In [788]:
# Import dependencies

import os
import json
import numpy as np
import pandas as pd

In [789]:
# Create references to the JSON files

purchase_json1 = "purchase_data.json"
purchase_json2 = "purchase_data2.json"

In [790]:
# Read with Pandas

purchase1_df = pd.read_json(purchase_json1)
purchase2_df = pd.read_json(purchase_json2)

# Same columns, so concatenate to tack DF1 onto DF2

frames = [purchase1_df, purchase2_df]
combined_df = pd.concat(frames)
combined_df.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


# Player Count

In [791]:
# Count number of uniques in SN column for number of player accounts

player_count= len(combined_df["SN"].unique())

player_count_df = pd.DataFrame({"Player Count":[player_count]})
player_count_df

,Player Count
0,612


# Purchasing Analysis (Total)

In [792]:
# Number of Unique Items
item_count = combined_df["Item Name"].nunique()

# Sum 'Price' column for Revenue
revenue = combined_df["Price"].sum()
revenue_formatted = '${:,.2f}'.format(revenue) 

# Count rows for number of purchases
rows = len(combined_df.index)

# Calculate average price by dividing price total over rows
average_price = revenue/rows
average_price_formatted ='${:,.2f}'.format(average_price)


# Create summary dictionary and populate dataframe
purchasing_analysis_df = pd.DataFrame({"Total Revenue":[revenue_formatted],
                                       "Number of Purchases":[rows],
                                       "Average Price":[average_price_formatted],
                                       "Number of Unique Items":[item_count]})


# Configure dataframe column order and display
purchasing_analysis_df = purchasing_analysis_df[['Number of Unique Items', 
                                                 'Average Price', 
                                                 'Number of Purchases', 
                                                 'Total Revenue' ]]
purchasing_analysis_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,180,$2.93,858,"$2,514.43"


# Gender Demographics

In [793]:
# Count number of unique gender members based on screenname

gender_nun = combined_df.groupby('Gender')['SN'].nunique()

count_male = gender_nun['Male']
count_female = gender_nun['Female']
count_other = gender_nun['Other / Non-Disclosed']

percent_male = '{0:.2f}'.format(((count_male/player_count) * 100))
percent_female = '{0:.2f}'.format(((count_female/player_count) * 100))
percent_other = '{0:.2f}'.format(((count_other/player_count) * 100))

In [794]:
# Create summary dictionary and populate dataframe

gender_dict = {"Gender": ['Male', 'Female', 'Other / Non-Disclosed'],
               "Percentage of Players": [percent_male, percent_female, percent_other],
               "Total Count": [count_male, count_female, count_other]
              }

gender_df = pd.DataFrame(gender_dict)

# Set row index to be Gender and display

gender_df = gender_df.set_index("Gender")
gender_df

,Percentage of Players,Total Count
Gender,,
Male,81.37,498
Female,18.30,112
Other / Non-Disclosed,1.47,9


# Purchasing Analysis (Gender)

In [795]:
# Group purchases by Gender in new dataframe

gender_purchases = combined_df.groupby("Gender")

In [796]:
# Count purchases by Gender

gender_purchases_counts = gender_purchases["Gender"].count()

In [797]:
# Find Average Purchase Price by Gender

average_gender_purchase = gender_purchases["Price"].mean()

In [798]:
# Find Total Purchase Value (Revenue) by Gender

total_gender_purchases = gender_purchases["Price"].sum()

In [799]:
# Find Normalized Totals (Total Purchase Value / Actual Gender Counts)

actual_player_counts_by_gender = [count_female, count_male, count_other]

normalized_gender_purchases = [t/g for t,g in zip(total_gender_purchases, actual_player_counts_by_gender)]

In [800]:
# Format appropriate calculated lists to currency

average_gender_purchase = ["$%.2f" % member for member in average_gender_purchase]
total_gender_purchases = ["$%.2f" % member for member in total_gender_purchases]
normalized_gender_purchases = ["$%.2f" % member for member in normalized_gender_purchases]

In [801]:
# Create summary dictionary and populate dataframe

gender_purchases_dict = {"Gender": ['Male', 'Female', 'Other / Non-Disclosed'],
                         "Purchase Count": gender_purchases_counts,
                         "Average Purchase Price": average_gender_purchase,
                         "Total Purchase Value": total_gender_purchases,
                         "Normalized Totals": normalized_gender_purchases
                        }

gender_purchases_df = pd.DataFrame(gender_purchases_dict)

# Set row index to Gender
gender_purchases_df = gender_purchases_df.set_index("Gender")


# Configure dataframe column order and display
gender_purchases_df = gender_purchases_df[["Purchase Count",
                                          "Average Purchase Price",
                                          "Total Purchase Value",
                                          "Normalized Totals"]]

gender_purchases_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Gender,,,,
Male,149,$2.85,$424.29,$3.79
Female,697,$2.94,$2052.28,$4.12
Other / Non-Disclosed,12,$3.15,$37.86,$4.21


# Age Demographics

In [802]:
# Create age bins and corresponding labels

age_bins = [0, 9, 14, 19, 24, 29, 34, 39, 150]
age_labels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]


# Bin the Age column

combined_df["Age Group"] = pd.cut(combined_df["Age"], age_bins, labels = age_labels)

In [803]:
# Create new dataframe grouped by Age Group

grouped_age_df = combined_df.groupby("Age Group")

In [804]:
# Count unique ages based on SN

age_nun = grouped_age_df["SN"].nunique()

In [805]:
# Calculate percentage of players represented by each age group and store in a list

age_bin_percentages = [((member/player_count)*100) for member in age_nun]
age_bin_percentages = ['%.2f' % member for member in age_bin_percentages ]

In [806]:
# Count number of unique players represented by age group

age_bin_counts = [age_nun[member] for member in age_labels]

In [807]:
# Create summary dictionary and populate dataframe

age_demo_dict = {"Ages": age_labels,
                 "Percentage of Players": age_bin_percentages,
                 "Total Count": age_bin_counts
                }

age_demo_df = pd.DataFrame(age_demo_dict)

# Set row index
age_demo_df = age_demo_df.set_index("Ages")

# Configure dataframe column order and display
age_demo_df = age_demo_df[["Percentage of Players","Total Count"]]

age_demo_df

,Percentage of Players,Total Count
Ages,,
<10,3.92,24
10-14,4.08,25
15-19,17.97,110
20-24,46.24,283
25-29,15.36,94
30-34,8.66,53
35-39,5.39,33
40+,1.96,12


# Purchasing Analysis (Age)

In [808]:
# Count purchases by Age Group

age_purchases_counts = grouped_age_df["Age Group"].count()

In [809]:
# Find Average Purchase Price by Age Group

average_age_purchase = grouped_age_df["Price"].mean()

In [810]:
# Find Total Purchase Value (Revenue) by Age Group

total_age_purchases = grouped_age_df["Price"].sum()

In [811]:
# Find Normalized Totals (Total Purchase Value / Unique Players by Age)

normalized_age_purchases = [t/g for t,g in zip(total_age_purchases, age_bin_counts)]

In [812]:
# Format appropriate calculated lists to currency

average_age_purchase = ["$%.2f" % member for member in average_age_purchase]
total_age_purchases = ["$%.2f" % member for member in total_age_purchases]
normalized_age_purchases = ["$%.2f" % member for member in normalized_age_purchases]

In [813]:
# Create summary dictionary and populate dataframe

age_purchases_dict = {"Ages": age_labels,
                         "Purchase Count": age_purchases_counts,
                         "Average Purchase Price": average_age_purchase,
                         "Total Purchase Value": total_age_purchases,
                         "Normalized Totals": normalized_age_purchases
                        }

age_purchases_df = pd.DataFrame(age_purchases_dict)

# Configure dataframe column order and display

age_purchases_df = age_purchases_df[["Purchase Count",
                                     "Average Purchase Price",
                                     "Total Purchase Value",
                                     "Normalized Totals"]]

age_purchases_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Age Group,,,,
<10,33,$2.95,$97.28,$4.05
10-14,38,$2.79,$105.91,$4.24
15-19,144,$2.89,$416.83,$3.79
20-24,372,$2.92,$1087.66,$3.84
25-29,134,$2.96,$396.44,$4.22
30-34,71,$2.97,$211.14,$3.98
35-39,48,$2.93,$140.77,$4.27
40+,18,$3.24,$58.40,$4.87


# Top Spenders

In [847]:
# Create new dataframe grouped by SN

spender_df = combined_df.groupby("SN")

In [848]:
# Find total purchases by SN

SN_total = spender_df.sum()["Price"]

In [849]:
# Find average purchase per SN

average_SN_purchase = spender_df["Price"].mean()

In [850]:
# Count number of purchases per SN

SN_purchases_counts = spender_df["Price"].count()

In [851]:
# Create summary dictionary and populate dataframe

SN_purchases_dict = {"Purchase Count": SN_purchases_counts,
                         "Average Purchase Price": average_SN_purchase,
                         "Total Purchase Value": SN_total
                        }

SN_purchases_df = pd.DataFrame(SN_purchases_dict)

# Configure dataframe column order and display
SN_purchases_df = SN_purchases_df[["Purchase Count",
                                          "Average Purchase Price",
                                          "Total Purchase Value"]]
# Sort by descending purchase value
SN_purchases_df = SN_purchases_df.sort_values("Total Purchase Value", ascending=False)

# Format to currency
SN_purchases_df["Average Purchase Price"] = SN_purchases_df["Average Purchase Price"].map("${:,.2f}".format)
SN_purchases_df["Total Purchase Value"] = SN_purchases_df["Total Purchase Value"].map("${:,.2f}".format)

# Display top 5 spender SNs
SN_purchases_df.head(5)

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Undirrala66,5,$3.41,$17.06
Aerithllora36,4,$3.77,$15.10
Saedue76,4,$3.39,$13.56
Sondim43,4,$3.25,$13.02
Mindimnya67,4,$3.18,$12.74


# Most Popular Items

In [852]:
# Collect item information

item_info = combined_df.loc[:,["Item ID", "Item Name", "Price"]]

In [853]:
# Calculate purchase count, item price, and total purchase value

total_item_purchase = item_info.groupby(["Item ID", "Item Name"]).sum()["Price"]
average_item_purchase = item_info.groupby(["Item ID", "Item Name"]).mean()["Price"]
item_purchase_count = item_info.groupby(["Item ID", "Item Name"]).count()["Price"]

In [854]:
# Format to currency

formatted_average_item_purchase = ["$%.2f" % member for member in average_item_purchase]
formatted_total_item_purchase = ["$%.2f" % member for member in total_item_purchase]

# Create dataframe

item_purchases_df = pd.DataFrame({"Purchase Count": item_purchase_count,
                                 "Item Price": formatted_average_item_purchase,
                                 "Total Purchase Value": formatted_total_item_purchase})

# Configure dataframe column order
item_purchases_df = item_purchases_df[["Purchase Count",
                                          "Item Price",
                                          "Total Purchase Value"]]

# Display top 5 items as measured by count
item_purchases_df.sort_values("Purchase Count", ascending=False).head(5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
84,Arcane Gem,12,$2.45,$29.34
39,"Betrayal, Whisper of Grieving Widows",11,$2.35,$25.85
31,Trickster,10,$2.32,$23.22
44,Bonecarvin Battle Axe,9,$2.67,$24.04
154,Feral Katana,9,$2.62,$23.55


# Most Profitable Items

In [855]:
# Collect item information

item_info = combined_df.loc[:,["Item ID", "Item Name", "Price"]]

# Calculate purchase count, item price, and total purchase value

total_item_purchase = item_info.groupby(["Item ID", "Item Name"]).sum()["Price"]
average_item_purchase = item_info.groupby(["Item ID", "Item Name"]).mean()["Price"]
item_purchase_count = item_info.groupby(["Item ID", "Item Name"]).count()["Price"]


In [856]:
# Create dataframe

item_purchases_df = pd.DataFrame({"Purchase Count": item_purchase_count,
                                 "Item Price": average_item_purchase,
                                 "Total Purchase Value": total_item_purchase})

# Configure dataframe column order and display
item_purchases_df = item_purchases_df[["Purchase Count",
                                          "Item Price",
                                          "Total Purchase Value"]]

item_purchases_df = item_purchases_df.sort_values("Total Purchase Value", ascending=False)


# Format to currency
item_purchases_df["Item Price"] = item_purchases_df["Item Price"].map("${:,.2f}".format)
item_purchases_df["Total Purchase Value"] = item_purchases_df["Total Purchase Value"].map("${:,.2f}".format)

# Display top 5 items by revenue
item_purchases_df.head(5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
34,Retribution Axe,9,$4.14,$37.26
107,"Splitter, Foe Of Subtlety",9,$3.67,$33.03
115,Spectral Diamond Doomblade,7,$4.25,$29.75
32,Orenmir,6,$4.95,$29.70
84,Arcane Gem,12,$2.45,$29.34
